In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import polars as pl
import toml
from support import preprocess
from scipy.interpolate import interp1d

In [27]:
_base_pth = toml.load("data_path.toml")['files']['pth']
_folder_name = 'normal_ss_t1'

_data_pth = os.path.join(_base_pth, _folder_name)

_mobbo_base_names = [
    'mobbo1.csv',
    'mobbo2.csv',
    'mobbo3.csv',
    'mobbo4.csv',
    'mobbo5.csv',
    'mobbo6.csv',
                     ]

In [28]:
dfs = {'m1':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[0])),
       'm2':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[1])),
       'm3':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[2])),
       'm4':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[3])),
       'm5':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[4])),
       'm6':pl.read_csv(os.path.join(_data_pth, _mobbo_base_names[5]))}

## Changing polarity for two boards m5 and m6

In [29]:
dfs['m5'] = dfs['m5'].with_columns([(pl.col('f1')*-1).alias('f1'),
                                    (pl.col('f2')*-1).alias('f2'),
                                    (pl.col('f3')*-1).alias('f3'),
                                    (pl.col('f4')*-1).alias('f4'),])

dfs['m6'] = dfs['m6'].with_columns([(pl.col('f1')*-1).alias('f1'),
                                    (pl.col('f2')*-1).alias('f2'),
                                    (pl.col('f3')*-1).alias('f3'),
                                    (pl.col('f4')*-1).alias('f4'),])

## Pre processing files

In [30]:
# trunkate dfs
_temp_df = dfs['m1']
_start_inx = None
_end_inx = None
_trailing_trigger = False
for i in range(len(_temp_df['sync'])):
    if (_temp_df['sync'][i] == 1) and (not _trailing_trigger):
        _start_inx = i
        _trailing_trigger = True
    
    if (_temp_df['sync'][i] == 0) and (_trailing_trigger):
        _end_inx = i
        break
    
_temp_df = _temp_df[_start_inx:_end_inx]       
        

In [34]:
def interpolate(ref_df, target_df):
    # df is the df to be interpolated
    # ref_df is the reference df
    # df and ref_df should have the same columns

    _reference_time = ref_df['timestamp']

    columns = target_df.columns[1:-1]
    interpolated_cols = {}
    for _col in columns:
        interp_func = interp1d(target_df['timestamp'], target_df[_col], kind='linear', axis=0, fill_value='extrapolate')
        interpolated_cols[_col] = interp_func(_reference_time)
        
    df = pl.from_dict(interpolated_cols).insert_column(0, _reference_time)
    return df

def interpolats_dfs(dfs, ref_no = 0):
    # generally first df is the reference df
    _keys = list(dfs.keys())
    _reference_df = dfs[_keys[ref_no]]
    _temp_dfs = dfs.copy()
    _temp_dfs.pop(_keys[ref_no])
    _selected_keys = _temp_dfs.keys()    
    _new_dfs = {}
    for _key in _selected_keys:
        _new_dfs[_key] = interpolate(_reference_df, dfs[_key]) 
        
    _new_dfs[_keys[ref_no]] = _reference_df
    
    return _new_dfs

In [35]:
dfs = interpolats_dfs(dfs)

In [37]:
dfs.keys()

dict_keys(['m2', 'm3', 'm4', 'm5', 'm6', 'm1'])